# 금융감독원 전자공시(DART) 크롤링

## 전자공시 웹페이지

http://dart.fss.or.kr/dsac001/search.ax

* selectDate : 날짜
* Cookie : DSAC001_MAXRESULTS=100;     
* 전자공시 목록을 조회하려면 Cookie를 사용해야 한다...!

![img](./images/05-1.jpg)

## wget을 사용하여 html 파일 다운받기

* https://eternallybored.org/misc/wget/
* 윈도우 64비트용 다운로드
* Anaconda3 폴더에 wget64.exe 파일 복사

In [1]:
! wget64 http://dart.fss.or.kr/dsac001/search.ax?selectDate=20170405 \
--header="Cookie:DSAC001_MAXRESULTS=5000;" \
-O DART-20170405.html

--2017-04-07 16:58:50--  http://dart.fss.or.kr/dsac001/search.ax?selectDate=20170405
Resolving dart.fss.or.kr (dart.fss.or.kr)... 61.73.60.200
Connecting to dart.fss.or.kr (dart.fss.or.kr)|61.73.60.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'DART-20170405.html'

     0K .......... .......... .......... .......... ..........  796K
    50K .......... .......... .......... .......... .......... 2.58M
   100K .......... .......... .......... .......... .......... 3.81M
   150K .......... .......... .......... .......... .......... 3.83M
   200K .......... .......... .......... .......... .......... 3.49M
   250K .......... .......... .......... .......... .......... 2.34M
   300K .......... .......... .......... .......... .......... 4.48M
   350K .......... .......... .......... .......... .......... 3.10M
   400K .......... .......... .......... .......... .......... 2.58M
   450K .......... .......... .......... ......

## request 를 이용하여 wget 함수 만들기

In [2]:
import requests
import os
from datetime import datetime, timedelta

In [3]:
data_dir = './dart_html/'   # 파일 저장할 폴더

In [4]:
url_templete = 'http://dart.fss.or.kr/dsac001/search.ax?selectDate=%s'
headers = {'Cookie':'DSAC001_MAXRESULTS=5000;'}

In [5]:
# wget 함수

In [6]:
def wget(url, local_filename):
    r = requests.get(url, headers=headers, stream=True)
    f = open(local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=1024): 
        if chunk:
            f.write(chunk)
            f.flush()
    return local_filename

In [7]:
# 크롤링할 날짜 범위
start_day = datetime(2017, 4, 1)
end_day = datetime(2017, 4, 5)
delta = end_day - start_day

In [8]:
for i in range(delta.days + 1):
    d = start_day + timedelta(days=i)        # 1일씩 증가
    rdate = d.strftime('%Y%m%d')
    filename = 'DART-' + rdate + '.html'     # 저장할 로컬 파일명
    wget(url_templete % rdate, data_dir + filename)   # 크롤링
    fsize = os.path.getsize(data_dir + filename) / (1024 ** 2)
    print ("%s (%.2f MB) downloaded" % (filename, fsize))

DART-20170401.html (0.00 MB) downloaded
DART-20170402.html (0.00 MB) downloaded
DART-20170403.html (2.59 MB) downloaded
DART-20170404.html (2.08 MB) downloaded
DART-20170405.html (2.26 MB) downloaded


## html 분석

![img](./images/05-2.jpg)

In [9]:
from bs4 import BeautifulSoup

In [16]:
fname = "./dart_html/DART-20170405.html"

with open(fname, encoding='utf-8') as f:
    text = f.read()
    soup = BeautifulSoup(text, "lxml")

In [17]:
# body-div-p-b
sel_list = soup.select('div p b')
counts = 0
count_str = sel_list[0].text
count_str

'\n\n\t\t전체\n\t\t \xa02,241건\n\n\t\t\t\t\t(2017년 04월 05일)\n\n\t\t\n\t'

In [18]:
count_str = "".join(count_str.split())
count_str

'전체2,241건(2017년04월05일)'

![img](./images/05-3.jpg)

In [19]:
# table
table = soup.find('table')
trs = table.findAll('tr')
print ("총 건수:", len(trs))

총 건수: 2242


In [20]:
# 0 (첫번째)는 테이블 헤더이므로 제외.
for tr in trs[1:10]:
    tds = tr.findAll('td')
    print(tds[0].text.strip(), end=" " )          # 시간
    print(tds[1].text.strip(), end=" " )          # 공시대상회사
    print(tds[1].img['alt'])                      # 이미지 태그 (유가증권, 코스닥, 코넥스 등)
    print( " ".join(tds[2].text.split()) )
    print( 'http://dart.fss.or.kr/' + tds[2].a['href'] )   # 링크
    print( tds[3].text.strip(), end=" ")                   # 제출인
    print( tds[4].text.strip().replace('.', '-') )         # 접수일자
    print()

18:29 SK가스 유가증권시장
증권신고서(채무증권)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405002607
SK가스 2017-04-05

18:11 오백볼트 코넥스시장
[기재정정]감사보고서제출
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405601087
오백볼트 2017-04-05

18:06 우리카드이천십칠의일유동화전문유한회사 기타법인
[기재정정]자산양도등의등록신청서
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170317000629
우리카드 2017-03-17

18:06 디비에르메스유동화전문유한회사 기타법인
[첨부추가]자산양도등의등록신청서
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170322000224
동부증권 2017-03-22

17:59 스테코 기타법인
감사보고서 (2016.12)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405002553
삼일회계법인 2017-04-05

17:59 마이크로텍 기타법인
감사보고서 (2016.12)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405002551
회계법인정연 2017-04-05

17:59 울산꿈나무 기타법인
감사보고서 (2016.12)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405002550
서린회계법인 2017-04-05

17:59 제다하네스 기타법인
감사보고서 (2016.12)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=20170405002549
팔공공인회계사감사반(제206호) 2017-04-05

17:59 이화에스피에스 기타법인
감사보고서 (2016.12)
http://dart.fss.or.kr//dsaf001/main.do?rcpNo=